In [1]:
import sys
import pandas as pd
sys.path.insert(0, '..')
from src.eventstream import Eventstream, EventstreamSchema, RawDataSchema
from src.tooling.step_matrix import StepMatrix

/home/julia.ostanina/retentioneering-tools-new-arch/examples/../src/params_model/params_model.py:68: UserWarning: Callable filter was excluded from schema since JSON schema has no equivalent type.
  params_schema: dict[str, Any] = cls.schema()
/home/julia.ostanina/retentioneering-tools-new-arch/examples/../src/params_model/params_model.py:68: UserWarning: Callable negative_function was excluded from schema since JSON schema has no equivalent type.
  params_schema: dict[str, Any] = cls.schema()
/home/julia.ostanina/retentioneering-tools-new-arch/examples/../src/params_model/params_model.py:68: UserWarning: Callable positive_function was excluded from schema since JSON schema has no equivalent type.
  params_schema: dict[str, Any] = cls.schema()


In [ ]:
1) Шаги
    1. Количество шагов у каждого пользователя в датасете [2, 100]
    2. Максимальное количество шагов в матрице [0, 2, 16, 50, 100, 200]
    3. Дробное число шагов? Отрицательное? ("ValueError: max() arg is an empty sequence")
2) Пороги
    1. Отрицательные, целые, ноль, один и т.д. [-50, -0.02, 0, 0.1, 0.05, 1, 67, 1000]
3)

In [2]:
# базовый датасет в формате датафрейма
raw_data = pd.read_csv("../src/datasets/data/simple-onlineshop.csv")

In [26]:
#пользователи отсортированные по количеству cобытий
raw_data.groupby(by='user_id', as_index=False).count().sort_values(by=['event'])

,user_id,event,timestamp
3750,999941967,2,2
447,132638296,2,2
1927,517179956,2,2
1941,519785837,2,2
1944,521074614,2,2
...,...,...,...
2881,760510988,83,83
265,78179456,83,83
3377,898482504,92,92
3621,962952209,94,94


In [30]:
min_table = raw_data.loc[raw_data['user_id'] == 999941967] #пользователь с двумя событиями

In [31]:
minmax_table = min_table.append(raw_data.loc[raw_data['user_id'] == 530349536], ignore_index = True) # минимум и максимум вместе 
minmax_table

,user_id,event,timestamp
0,999941967,catalog,2020-04-17 12:18:49.391199
1,999941967,lost,2020-04-17 12:18:50.391199
2,530349536,catalog,2019-11-12 10:03:06.441295
3,530349536,cart,2019-11-12 10:03:34.490371
4,530349536,delivery_choice,2019-11-12 10:03:36.538909
...,...,...,...
97,530349536,main,2020-03-15 03:08:17.586834
98,530349536,catalog,2020-03-15 03:08:18.914158
99,530349536,cart,2020-03-15 03:08:23.925717
100,530349536,delivery_choice,2020-03-15 03:08:27.382559


In [33]:
# create data schema
raw_data_schema = RawDataSchema(
    event_name="event", event_timestamp="timestamp", user_id="user_id")

# create source eventstream
source = Eventstream(
    raw_data=minmax_table,
    raw_data_schema=raw_data_schema,
    schema=EventstreamSchema()
)

In [ ]:
max_step_range = [0, 2, 16, 50, 100, 200]

In [47]:
step_matrix = StepMatrix(eventstream=source, 
                          max_steps=15)
type(step_matrix._get_plot_data())

tuple

In [44]:
source.step_matrix(max_steps=16,
          thresh=0.05)

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [45]:
source_event123 = pd.DataFrame(
            [
                [1, "event1", "2022-01-01 00:01:00"],
                [1, "event2", "2022-01-01 00:01:02"],
                [1, "event1", "2022-01-01 00:02:00"],
                [1, "event1", "2022-01-01 00:03:00"],
                [1, "event1", "2022-01-01 00:03:00"],
                [1, "event3", "2022-01-01 00:03:30"],
                [1, "event1", "2022-01-01 00:04:00"],
                [1, "event3", "2022-01-01 00:04:30"],
                [1, "event1", "2022-01-01 00:05:00"],
                [2, "event1", "2022-01-02 00:00:00"],
                [2, "event2", "2022-01-02 00:00:05"],
                [2, "event2", "2022-01-02 00:01:05"],
                [3, "event1", "2022-01-02 00:01:10"],
                [3, "event1", "2022-01-02 00:02:05"],
                [3, "event4", "2022-01-02 00:03:05"],
                [4, "event1", "2022-01-02 00:01:10"],
                [4, "event1", "2022-01-02 00:02:05"],
                [4, "event1", "2022-01-02 00:03:05"],
            ],
            columns=["user_id", "event", "timestamp"],
        )
source_event123

,user_id,event,timestamp
0,1,event1,2022-01-01 00:01:00
1,1,event2,2022-01-01 00:01:02
2,1,event1,2022-01-01 00:02:00
3,1,event1,2022-01-01 00:03:00
4,1,event1,2022-01-01 00:03:00
5,1,event3,2022-01-01 00:03:30
6,1,event1,2022-01-01 00:04:00
7,1,event3,2022-01-01 00:04:30
8,1,event1,2022-01-01 00:05:00
9,2,event1,2022-01-02 00:00:00
